In [1]:
from InstagramAPI import InstagramAPI
import time
from datetime import datetime
import requests
import json
import pandas as pd

In [2]:
url = 'https://www.instagram.com/p/BoHGsQ6HJ6W/'

In [3]:
res = requests.get('https://api.instagram.com/oembed/?callback=&url='+url)

In [4]:
json_res = json.loads(res.text)

In [5]:
user_id = json_res['author_id']

In [6]:
akun = open('akun.env', 'r').read().split('\n')

In [7]:
API = InstagramAPI(akun[0], akun[1])
API.login()

Request return 400 error!
Two Factor Authentication Required!
Two Factor Identifier:  VgkqTpEhbH
Finish login with finishTwoFactorAuth(verification_code, two_factor_identifier) !


In [8]:
API.finishTwoFactorAuth('052713', 'VgkqTpEhbH')

Request return 404 error!
Login success with two factor authentication!



True

In [9]:
more_available = True
feeds = []
max_feed_id = ''

In [10]:
while more_available:
    feed_seva = API.getUserFeed(user_id, maxid=max_feed_id)
    # feed' page come from older to newer, lets preserve desc order in full list
    for c in API.LastJson['items']:
        feeds.append(c)
    more_available = API.LastJson.get('more_available', False)
    # next page
    if more_available:
        max_feed_id = API.LastJson.get('next_max_id', '')
        time.sleep(2)

In [11]:
feeds1 = []
for feed in feeds:
#     print(feed)
    temp = {}
    temp['user_id'] = feed['user']['pk']
    temp['username'] = feed['user']['username']
    temp['media_id'] = feed['id']
    temp['comment_count'] = feed['comment_count']
    temp['like_count'] = feed['like_count']
    temp['caption'] = feed['caption']['text'] if feed['caption'] else ''
#     temp['url_images'] = feed['image_versions2']['candidates'][0]['url']
    temp['unique_code'] = feed['code']
    temp['taken_at'] = feed['taken_at']
    
    feeds1.append(temp)

In [12]:
feeds_seva = pd.DataFrame(feeds1)

In [13]:
feeds_seva.head()

,caption,comment_count,like_count,media_id,taken_at,unique_code,user_id,username
0,FLASH NEWS !\n•\nDaihatsu All New Terios dibel...,274,490,1917574694065029692_6030823779,1542812738,BqcmAZGBkI8,6030823779,sevaid_official
1,Jangan lewatkan Flash Sale Mobil yang ke-3 dar...,416,362,1917433559788788050_6030823779,1542795930,BqcF6njn5lS,6030823779,sevaid_official
2,Malam ini Flash Sale New Terios mulai pukul 21...,256,621,1917350417314884630_6030823779,1542786002,BqbzAvGACwW,6030823779,sevaid_official
3,Hanya tinggal 1 hari menuju Flash Sale Terios ...,132,427,1916454787113727304_6030823779,1542679235,BqYnXmeArVI,6030823779,sevaid_official
4,Ayo share topik menarik yang ingin kamu bahas ...,27,160,1916062078263988520_6030823779,1542632420,BqXOE72hZEo,6030823779,sevaid_official


In [23]:
len(comments)

10332

In [16]:
comments = []
for feed in feeds1:
    has_more_comments = True
    max_id = ''
    while has_more_comments:
        _ = API.getMediaComments(feed['media_id'], max_id=max_id)
        # comments' page come from older to newer, lets preserve desc order in full list
        for c in reversed(API.LastJson['comments']):
            temp_com = {}
            temp_com['comment_id'] = c['pk']
            temp_com['user_id_comment'] = c['user_id']
            temp_com['username_coment'] = c['user']['username']
            temp_com['media_id'] = feed['media_id']
            temp_com['text'] = c['text']
            temp_com['created_at'] = c['created_at']
            
            comments.append(temp_com)
        has_more_comments = API.LastJson.get('has_more_comments', False)
        # next page
        if has_more_comments:
            max_id = API.LastJson.get('next_max_id', '')
            time.sleep(2)

Except on SendRequest (wait 60 sec and resend): HTTPSConnectionPool(host='i.instagram.com', port=443): Max retries exceeded with url: /api/v1/media/1875544300446364809_6030823779/comments/?max_id=%7B%22server_cursor%22:%20%22QVFDSkdmMERzVEtrVmNETjc5TFh0ZkJOMU5BeEE0OW1LM1gyQ0xueG9TOFBFYmxOWFBHZnRoUWEwaU9saTJMMDFYNXcyRnZ3bEIwdXJjaXNFMk10eERNXw==%22,%20%22is_server_cursor_inverse%22:%20false%7D (Caused by NewConnectionError('<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x000001FC12E48438>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond',))


In [24]:
comments_seva = pd.DataFrame(comments)

In [32]:
feeds_seva.to_csv('feedseva20181122.csv')

In [33]:
comments_seva.to_csv('commentseva20181122.csv')

In [25]:
from datetime import datetime

In [28]:
comments_seva['created_at'] = comments_seva['created_at'].apply(lambda x: datetime.strftime(datetime.utcfromtimestamp(x), '%Y-%m-%d %H:%I:%S'))

TypeError: an integer is required (got type str)

In [29]:
feeds_seva['taken_at'] = feeds_seva['taken_at'].apply(lambda x: datetime.strftime(datetime.utcfromtimestamp(str(x)), '%Y-%m-%d %H:%I:%S'))

TypeError: an integer is required (got type str)

In [30]:
comments_seva.tail()

,comment_id,created_at,media_id,text,user_id_comment,username_coment
10327,17913450301083706,2018-03-19 13:01:23,1738465428675687504_6030823779,"Mobil impian yg pengen dibeli Alphard, biar bi...",5451372414,elsavinietta
10328,17930611246034869,2018-03-19 12:12:14,1738465428675687504_6030823779,Wah pas bgt nih.. dr dulu ngidam mu-X gara2 di...,257532447,irham_n
10329,17913283954085809,2018-03-19 12:12:04,1738465428675687504_6030823779,milih isuzu mu-X lah.. alasan nya... woiii yan...,2125715332,astragemaislami
10330,17931908614031152,2018-03-20 06:06:23,1726077600039044322_6030823779,Bbnjjkllioop,4237703022,dinadinar_booster
10331,17930804395204963,2018-10-19 23:11:26,1723120736020094492_6030823779,Bismillah Done spam like @sevaid_official 😍 se...,1537943551,hense246


In [31]:
feeds_seva.tail()

,caption,comment_count,like_count,media_id,taken_at,unique_code,user_id,username
166,Hari gini masih rebutan untuk cari parkir? Ber...,0,65,1725312991086109474_6030823779,2018-03-01 08:08:23,BfxiwY4DeMi,6030823779,sevaid_official
167,Mau cari otomotif terbaru tanpa buang waktu da...,0,65,1725311345350239558_6030823779,2018-03-01 08:08:19,BfxiYcKjWVG,6030823779,sevaid_official
168,"Jika sudah ada yang terpercaya, kenapa harus a...",0,60,1723150954495685058_6030823779,2018-02-26 09:09:22,Bfp3KnZFzHC,6030823779,sevaid_official
169,"Kalau bisa disatu tempat, kenapa harus yang be...",0,54,1723135821379667135_6030823779,2018-02-26 08:08:18,BfpzuZlFgS_,6030823779,sevaid_official
170,"Daripada nunggu yang lama, mending cari yang l...",1,53,1723120736020094492_6030823779,2018-02-26 08:08:20,BfpwS4Pll4c,6030823779,sevaid_official


In [6]:
from InstagramAPI import InstagramAPI
 
api = InstagramAPI("stezarpriansya@gmail.com", "Stezar-123!")     #Used actual username and password. Not this!
 
login = False
 
if(api.login()):
    login = True
elif api.LastJson['two_factor_required'] == True:
    tfa_id = api.LastJson['two_factor_info']['two_factor_identifier']
    tfa_sms_code = input("Enter 2fa sms code: ")
 
    print(tfa_id, tfa_sms_code)
 
    if api.finishTwoFactorAuth(tfa_sms_code, tfa_id) == True:
        login = True
    else:
        print("2FA Failed....")
        print(api.LastResponse)  # print last response
else:
    print("Some other login error")
    print(api.LastResponse)  # print last response
    exit()
 
 
if (login):
    api.getSelfUserFeed()  # get self user feed
    print(api.LastJson)  # print last response JSON
    print("Login succes!")
else:
    print("Can't login!")

Request return 400 error!
Two Factor Authentication Required!
Two Factor Identifier:  RK1eMhkiZW
Finish login with finishTwoFactorAuth(verification_code, two_factor_identifier) !
Enter 2fa sms code: 
RK1eMhkiZW 
Request return 400 error!
2FA Failed....
<Response [400]>
Can't login!
